In [1]:
from PromptOps.std_templates import ShotTemplateFormatter
from PromptOps.icqa_templates import ICQATemplateFormatter
from PromptOps.cot_templates import COTTemplateFormatter
from PromptOps.test import PromptCompletion, Test
from PromptOps.test_suite import TestSuite
from PromptOps.perturb import Perturbation

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chomm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
c:\Users\chomm\anaconda3\envs\sunghoon\lib\site-packages\huggingface_hub\file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Define LLM

In [2]:
# Initialize the PromptCompletion model for Llama
llama_url = "http://127.0.0.1:8000/v1/chat/completions"  # Replace with your Llama model URL
completion = PromptCompletion(
    model_provider="llama",
    model="llama-13b",
    system_content="You are an assistant that answer the question",
    temperature=0.5,
    top_p=0.9,
    max_tokens=150,
    llama_url=llama_url
)

In [3]:
perturbation = Perturbation()

## Function

In [28]:
import pandas as pd
# shot_type='zero', 'one', 'few'
def process_test_robust(file_path, shot_type, template):
    """
    Process the input CSV file, run tests on the data, and return index scores.

    Args:
        file_path (str): Path to the input CSV file.
        shot_type (str): Type of shot (e.g., 'one', 'few', etc.). Default is 'one'.

    Returns:
        dict: A dictionary containing scores for each Original_Question_Index.
    """
    # Load the CSV file
    df = pd.read_csv(file_path)

    # Process questions with perturbations
    result_df = perturbation.process_questions(df, question_column="Question", expected_answer_column="Expected_answer")
    columns_to_merge = df.drop(columns=["Question", "Expected_answer"])

    # Merge with additional columns
    merged_result_df = result_df.merge(
        columns_to_merge,
        left_on="Original_Question_Index",
        right_index=True,
        how="left"
    )

    # Rearrange column order
    column_order = ["Original_Question_Index"] + \
                   list(columns_to_merge.columns) + \
                   ["Original_Question", "Perturbation", "Perturbed_Question", "Expected_Answer"]
    merged_result_df = merged_result_df[column_order]

    # Save intermediate results
    output_file_path = "merged_perturbation_results.csv"
    merged_result_df.to_csv(output_file_path, index=False)

    # Format data for testing
    if template=='std':
        robust_formatter = ShotTemplateFormatter(output_file_path)
    elif template=='icqa':
        robust_formatter = ICQATemplateFormatter(output_file_path)
    elif template=='cot':
        robust_formatter = COTTemplateFormatter(output_file_path)

    robust_formatted_data = robust_formatter.format_all_rows(shot_type=shot_type, perturb_type='robust')
    robust_formatter.save_formatted_data_to_csv(robust_formatted_data, 'robust_new_formatted_one_shot.csv')
    robust_formatted_data = pd.DataFrame(robust_formatted_data)

    # Initialize TestSuite
    test_suite = TestSuite()
    index_scores = {}
    
    detailed_results = []  # Store detailed results for each test

    for original_index in robust_formatted_data['Original_Question_Index'].unique():
        print(f"Running [Original_Question_Index {original_index}] test...")

        subset = robust_formatted_data[robust_formatted_data['Original_Question_Index'] == original_index]
        index_result = {"Original_Question_Index": original_index, "tests": []}

        for _, row in subset.iterrows():
            test = Test(
                name=f"Test robust #{row.name + 1}",
                prompt=row["original_prompt"],
                expected_result=row["expected_result"],
                description=f"A test with robust perturbation",
                perturb_method=row["perturb_type"],
                test_type=row["perturb_type"],
                perturb_text=row["perturb_prompt"],
            )
            test_suite.add_test(test)

        test_suite.run_all(completion)
        results, summary = test_suite.summarize()
        total_tests = summary['total_tests']
        failures = summary['failures']
        score = (total_tests - failures) / total_tests * 100

        index_scores[original_index] = score
        index_result["score"] = score
        index_result["summary"] = summary
        index_result["results"] = results
        detailed_results.append(index_result)

        print(f"Summary: {summary}")
        print(f"Robust index {original_index}, score={score:.2f}%")
        test_suite.clear()
    
    return index_scores, detailed_results


In [29]:
def process_test(file_path, shot_type, template, perturbation_types):
    if template=='std':
        formatter = ShotTemplateFormatter(file_path)
    elif template=='icqa':
        formatter = ICQATemplateFormatter(file_path)
    elif template=='cot':
        formatter = COTTemplateFormatter(file_path)
    
    csv_files = []
    for perturb_type in perturbation_types:
        #print(f"Processing {perturb_type} perturbation...")
        formatted_data = formatter.format_all_rows(shot_type=shot_type, perturb_type=perturb_type)
        output_file = f"formatted_{perturb_type}_output.csv"

        formatter.save_formatted_data_to_csv(formatted_data, output_file)
        csv_files.append((perturb_type, output_file))
        #print(f"Saved formatted data with {perturb_type} perturbation to {output_file}")

    test_suite = TestSuite()
    for perturb_type, file_path in csv_files:
        #print(f"Processing file: {file_path} with perturbation: {perturb_type}")
        
        # Load the CSV into a DataFrame
        df = pd.read_csv(file_path)
        
        # Iterate over each row to create a test
        for index, row in df.iterrows():
            
            test = Test(
                name=f"Test {perturb_type} #{index + 1}",
                prompt=row["original_prompt"],  # Original prompt
                expected_result=row["expected_result"],  # Expected result
                description=f"A test with {perturb_type} perturbation",
                perturb_method=perturb_type,  # Perturbation type
                test_type=perturb_type,
                perturb_text=row["perturb_prompt"],  # Perturbed prompt
            )
            test_suite.add_test(test)
    test_suite.run_all(completion)
    results, summary = test_suite.summarize()

    test_suite.export_results("test_results.csv", file_format="csv", overwrite=True)
    test_suite.export_results("test_results.xlsx", file_format="xlsx", overwrite=True)
    test_suite.export_results("test_results.json", file_format="json", overwrite=True)

    test_suite.clear()
    return results, summary


In [30]:
def process_score(index_scores, summary):
    # Initialize counters
    overall_total_tests = 0
    overall_failures = 0

    # Threshold for failure
    threshold = 0.7

    # Process index scores
    for index, score in index_scores.items():
        overall_total_tests += 1  # Each index counts as one test
        if score < threshold * 100:  # Convert threshold to percentage
            overall_failures += 1

    # Add results from the summary for total tests and failures
    overall_total_tests += summary['total_tests']
    overall_failures += summary['failures']

    # Calculate overall failure rate
    overall_failure_rate = (overall_failures / overall_total_tests) * 100
    return overall_total_tests, overall_failures, overall_failure_rate
   

In [36]:
def calculate_performance_score(detailed_scores, results):
    import json

    # Extract all score values from detailed scores
    detailed_scores_values = []
    for score in detailed_scores:
        for result in score['results']:
            detailed_scores_values.append(result['score_original'])

    # Combine detailed scores and results scores
    all_scores = detailed_scores_values + [result['score_original'] for result in results]

    # Calculate the overall performance score
    overall_performance_score = sum(all_scores) / len(all_scores)

    # Initialize a dictionary to group scores by perturbation type
    perturbation_scores = {}

    for result in results:
        test_type = result.get('test_type') or result['name'].split("#")[0].strip().lower()
        if test_type not in perturbation_scores:
            perturbation_scores[test_type] = []
        perturbation_scores[test_type].append(result['score_original'])

    # Add robust scores from detailed scores to the group
    if 'robust' not in perturbation_scores:
        perturbation_scores['robust'] = []
    for detailed_score in detailed_scores:
        perturbation_scores['robust'].append(detailed_score['score'])

    # Calculate the average score for each perturbation type
    perturbation_averages = {}
    for perturbation, scores in perturbation_scores.items():
        perturbation_averages[perturbation] = sum(scores) / len(scores)

    # Combine the results into a JSON-friendly dictionary
    result_data = {
        "overall_performance_score": overall_performance_score,
        **perturbation_averages,
    }

    return json.dumps(result_data, indent=4)

## Run

### Robust
- index_scores (use in summary score)  -> calcutale failure rate / total test 
- detailed_results ((use in dashboard score) ) -> calculate performance dashboard

In [34]:
index_scores, robust_results = process_test_robust('one_shot_example.csv', shot_type='one', template='icqa')


Running [Original_Question_Index 0] test...
Summary: {'total_tests': 5, 'failures': 0, 'fail_rate': 0.0}
Robust index 0, score=100.00%
Running [Original_Question_Index 1] test...
Summary: {'total_tests': 8, 'failures': 0, 'fail_rate': 0.0}
Robust index 1, score=100.00%
Running [Original_Question_Index 2] test...
Summary: {'total_tests': 5, 'failures': 1, 'fail_rate': 20.0}
Robust index 2, score=80.00%


In [39]:
# Result
print("Total test:", index_scores)
print("Detailed score:", robust_results)
#{0: 100.0, 1: 100.0, 2: 100.0}
# 0 = '1st Prompt'
# 1 = '2nd Prompt'
# 2 = '3rd Prompt'

Total test: {0: 100.0, 1: 100.0, 2: 80.0}
Detailed score: [{'Original_Question_Index': 0, 'tests': [], 'score': 100.0, 'summary': {'total_tests': 5, 'failures': 0, 'fail_rate': 0.0}, 'results': [{'name': 'Test robust #1', 'description': 'A test with robust perturbation', 'test_type': 'robust', 'prompt': 'Instruction: Answer the following questions based on the context and provide answer in boolean (Yes or No)\nContext: Cucumber plants need insects to pollinate them. Seedless cucumber does not require pollination.\nQ: Is growing seedless cucumber good for a gardener with entomophobia?\nA: Yes\n\nQ: Can cucumbers grow without insects?\nA:', 'expected_result': 'Yes', 'perturb_text': 'Instruction: Answer the following questions based on the context and provide answer in boolean (Yes or No)\nContext: Cucumber plants need insects to pollinate them. Seedless cucumber does not require pollination.\nQ: Is growing seedless cucumber good for a gardener with entomophobia?\nA: Yes\n\nQ: aCn cucumbe

### Other perturbation

In [21]:
other_results, summary = process_test('one_shot_example.csv', shot_type='one', template='icqa', perturbation_types = ['taxonomy', 'vocab'])

- summary (use in summary score)  -> calcutale failure rate / total test 
- other_results ((use in dashboard score) ) -> calculate performance dashboard

In [44]:
print("Results:", other_results)
print("Summary:", summary)

Results: [{'name': 'Test taxonomy #1', 'description': 'A test with taxonomy perturbation', 'test_type': 'taxonomy', 'prompt': 'Instruction: Answer the following questions based on the context and provide answer in boolean (Yes or No)\nContext: Cucumber plants need insects to pollinate them. Seedless cucumber does not require pollination.\nQ: Is growing seedless cucumber good for a gardener with entomophobia?\nA: Yes\n\nQ: Can cucumbers grow without insects?\nA:', 'expected_result': 'Yes', 'perturb_text': 'Instruction: Answer the following questions based on the context and provide answer in boolean (Yes or No)\nContext: Cucumber plants need insects to pollinate them. Seedless cucumber does not require pollination.\nQ: Is growing seedless cucumber good for a gardener with entomophobia?\nA: Yes\n\nQ: Can cucumbers grow without insects ?\nA:', 'pass_condition': 'increase', 'capability': None, 'response_original': 'No', 'response_perturb': 'No', 'score_original': 0.7000373601913452, 'score

## Summary score of Robustness and other perturbation

In [37]:
overall_total_tests, overall_failures, overall_failure_rate = process_score(index_scores, summary)

# Display results
print(f"Overall Total Tests: {overall_total_tests}")
print(f"Overall Failures: {overall_failures}")
print(f"Overall Failure Rate: {overall_failure_rate:.2f}%")

Overall Total Tests: 9
Overall Failures: 0
Overall Failure Rate: 0.00%


## Dashboard Score

show performance


In [42]:
performance = calculate_performance_score(robust_results, other_results)

In [43]:
print(performance)
# overall performance score is average score original 

{
    "overall_performance_score": 0.7875264634688696,
    "taxonomy": 0.8000249067942301,
    "vocab": 0.8000249067942301,
    "robust": 93.33333333333333
}
